# **Juypter "GUI" using ipywidgets**

helpful links:

https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916

https://towardsdatascience.com/interactive-spreadsheets-in-jupyter-32ab6ec0f4ff

In [ ]:
import ipywidgets as widgets

import pandas as pd
import numpy as np

## **Filtering a list**
taken from https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916

In [ ]:
url = r"https://data.london.gov.uk/download/number-international-visitors-london/b1e0f953-4c8a-4b45-95f5-e0d143d5641e/international-visitors-london-raw.csv"
df_london = pd.read_csv(url, encoding = "ISO-8859-1")

In [ ]:
ALL = 'ALL'
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

In [ ]:
dropdown_year = widgets.Dropdown(options=unique_sorted_values_plus_ALL(df_london.year))

output_year = widgets.Output()

def dropdown_year_eventhandler(change):
    output_year.clear_output()        
    with output_year:
        if (change.new == ALL):
            display(df_london)
        else:
            display(df_london[df_london.year == change.new])
        
dropdown_year.observe(dropdown_year_eventhandler, names='value')
display(dropdown_year)

In [ ]:
display(output_year)

-----
# **Select file using a button and askopenfilename**

code from: https://codereview.stackexchange.com/questions/162920/file-selection-button-for-jupyter-notebook
        
small bugfix if no file selected

In [ ]:
import traitlets
from ipywidgets import widgets
from IPython.display import display
from tkinter import Tk, filedialog


class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self):
        super(SelectFilesButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        _files =filedialog.askopenfilename(multiple=True)
        if _files:
            b.files = _files

            b.description = "Files Selected"
            b.icon = "check-square-o"
            b.style.button_color = "green"
        else:
            b.files = []
            b.description = "File selection aborded"
            b.icon = "square-o"
            b.style.button_color = "red"

In [ ]:
my_button = SelectFilesButton()
my_button # This will display the button in the context of Jupyter Notebook

In [ ]:
# In a different cell of the same Jupyter Notebook You can access the file list by using the following:
my_button.files